<a href="https://colab.research.google.com/github/pszemraj/job-scraper/blob/master/switzerland/Job_Scraper_CH_Edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# job scraper (simple)

from [this article](https://towardsdatascience.com/automating-my-job-search-with-python-ee2b465c6a8f) on medium



# Set up

In [2]:
%%capture

!pip install -U selenium
!pip install -U bs4

from os.path import join
from google.colab import data_table
%load_ext google.colab.data_table


In [3]:
"""
Created on Tue Apr 28 11:35:04 2020

@author: chrislovejoy
"""


import urllib
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os


In [4]:

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## attempt to load driver

this is just for the other website originally used. For the original job scraper, this file is supposed to be in your working directory.. I tried to store on dropbox and then load it but doesn't work. May look into a better solution for Colab in the future, but for now it is a TODO (eventually)

In [5]:

driver_url = "https://www.dropbox.com/s/3e8kqk4c8hfcy83/chromedriver?dl=1"
r = requests.get(driver_url, allow_redirects=True)

# names = getFilename_fromCd(r.headers.get('content-disposition'))
# fixed_fnames = names.split(";") # split the multiple results
# this_filename = URL_string_filter(fixed_fnames[0])
this_filename = "chromedriver"
storage_loc = os.getcwd()
open(join(storage_loc, this_filename), 'wb').write(r.content)

14786468

# Job Search Functions


## generic

In [6]:
def save_jobs_to_excel(jobs_list, filename, verbose=False):
    jobs = pd.DataFrame(jobs_list)
    jobs.to_excel(filename)

    if verbose:
        print("saved the following to excel with filename {}: \n".format(filename))

        print(jobs.info())
    return jobs

## Swiss-specific

In [7]:
## added by Peter

## indeed switzerland -------------------------------------------------------
# website = "https://ch.indeed.com/Switzerland-English-Jobs"
# base ->
# example search https://ch.indeed.com/Stellen?q=Switzerland+English&jt=internship
# data job, language = EN, no others: https://ch.indeed.com/Stellen?q=data&lang=en
# ^ but internship https://ch.indeed.com/Stellen?q=data&jt=internship&lang=en

def load_indeed_jobs_CH(job_query, job_type=None, language=None, 
                        run_default=False):
    i_website = "https://ch.indeed.com/Stellen?"
    def_website = "https://ch.indeed.com/Stellen?q=Switzerland+English&jt=internship"
    if run_default:
        # switzerland has a unique page shown below, can run by default
        # website = "https://ch.indeed.com/Switzerland-English-Jobs"

        url = def_website
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        job_soup = soup.find(id="resultsCol")
    else:
        getVars = {'q' : job_query, 'jt' : job_type, 'lang' : language,
                'fromage' : 'last', 'sort' : 'date'}

        # if values are not specified, then remove them from the dict (and URL)
        if job_query is None:
            del getVars['q']
        if job_type is None:
            del getVars['jt']
        if language is None:
            del getVars['lang']

        url = (i_website + urllib.parse.urlencode(getVars))
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        job_soup = soup.find(id="resultsCol")

    # return the job soup

    soup_results = {
        "job_soup": job_soup,
        "query_URL": url

    }
    return soup_results




In [8]:
from datetime import date
import pprint as pp

today = date.today()
# Month abbreviation, day and year	
d4 = today.strftime("%b-%d-%Y")

default_filename = d4 + "JS_test_results.xls"

def find_CHjobs_from(website, desired_characs, job_query, job_type=None, 
                   language=None, filename=default_filename, verbose=False):    
    """
    This function extracts all the desired characteristics of all new job postings
    of the title and location specified and returns them in single file.
    The arguments it takes are:
        - Website: to specify which website to search (options: 'indeed' or 'indeed_default')
        - job_query: words that you want to narrow down the jobs to. for example 'data'
        - job_type: 'internship' or 'fulltime' or 'permanent'
        - language: 'en' or 'de' or leave blank
        - Desired_characs: this is a list of the job characteristics of interest,
            from titles, companies, links and date_listed.
        - Filename: default is "JS_test_results.xls", can be changed to whatever 
    """
    
    if website == 'indeed':
        sp_search = load_indeed_jobs_CH(job_query, 
                                       job_type=job_type, language=language)
        job_soup = sp_search.get("job_soup")
        URL_used = sp_search.get("query_URL")

        if verbose:
            print("\n The full HTML docs are: \n")
            pp.pprint(job_soup, compact=True)
        jobs_list, num_listings = extract_job_information_indeedCH(job_soup, 
                                                                   desired_characs,
                                                                   uURL=URL_used)
    elif website == 'indeed_default':
        job_soup = load_indeed_jobs_CH(job_query, run_default=True)
        jobs_list, num_listings = extract_job_information_indeedCH(job_soup, desired_characs)

    job_df = save_jobs_to_excel(jobs_list, filename)
 
    print('{} new job postings retrieved from {}. Stored in {}.'.format(num_listings, 
                                                                          website, filename))
    
    return job_df
        

## Indeed-specific

In [9]:
def_URL = "https://ch.indeed.com/Stellen?" + "ADD_queries_here" 

def extract_job_information_indeedCH(job_soup, desired_characs, uURL=def_URL):
    job_elems = job_soup.find_all('div', class_='jobsearch-SerpJobCard')
     
    cols = []
    extracted_info = []
    
    
    if 'titles' in desired_characs:
        titles = []
        cols.append('titles')
        for job_elem in job_elems:
            titles.append(extract_job_title_indeed(job_elem))
        extracted_info.append(titles)                    
    
    if 'companies' in desired_characs:
        companies = []
        cols.append('companies')
        for job_elem in job_elems:
            companies.append(extract_company_indeed(job_elem))
        extracted_info.append(companies)
    
    if 'links' in desired_characs:
        links = []
        cols.append('links')
        for job_elem in job_elems:
            links.append(extract_link_indeedCH(job_elem, uURL))
        extracted_info.append(links)
    
    if 'date_listed' in desired_characs:
        dates = []
        cols.append('date_listed')
        for job_elem in job_elems:
            dates.append(extract_date_indeed(job_elem))
        extracted_info.append(dates)
    
    jobs_list = {}
    
    for j in range(len(cols)):
        jobs_list[cols[j]] = extracted_info[j]
    
    num_listings = len(extracted_info[0])
    
    return jobs_list, num_listings


def extract_job_title_indeed(job_elem):
    title_elem = job_elem.find('h2', class_='title')
    title = title_elem.text.strip()
    return title

def extract_company_indeed(job_elem):
    company_elem = job_elem.find('span', class_='company')
    company = company_elem.text.strip()
    return company

def extract_link_indeedCH(job_elem, uURL):
    # working example https://ch.indeed.com/Stellen?q=data&jt=internship&lang=en&vjk=49ed864bd5e422fb

    link = job_elem.find('a')['href']
    uURL_list = uURL.split("&fromage=last")
    link = uURL_list[0] + "&" + link
    return link.replace("/rc/clk?jk=", "vjk=")

def extract_date_indeed(job_elem):
    date_elem = job_elem.find('span', class_='date')
    date = date_elem.text.strip()
    return date



# Main

## key settings

In [10]:
desired_characs = ['titles', 'companies', 'links', 'date_listed']

download_output = True

## Swiss Jobs - Indeed

```
    This function extracts all the desired characteristics of all new job postings
    of the title and location specified and returns them in single file.
    The arguments it takes are:
        - Website: to specify which website to search (options: 'indeed' or 'indeed_default')
        - job_query: words that you want to narrow down the jobs to. for example 'data'
        - job_type: 'internship' or 'fulltime' or 'permanent'
        - language: 'en' or 'de' or leave blank
        - Desired_characs: this is a list of the job characteristics of interest,
            from titles, companies, links and date_listed.
        - Filename: default is "JS_test_results.xls", can be changed to whatever 
```


In [11]:
# website = "https://ch.indeed.com/Switzerland-English-Jobs"
# base ->
# example search https://ch.indeed.com/Stellen?q=Switzerland+English&jt=internship
# data job, language = EN, no others: https://ch.indeed.com/Stellen?q=data&lang=en
# ^ but internship https://ch.indeed.com/Stellen?q=data&jt=internship&lang=en
# https://ch.indeed.com/Stellen?q=data&jt=internship&lang=en&vjk=2ffc5d47b54d9ed5
    
chdf1 = find_CHjobs_from(website="indeed", desired_characs=desired_characs, 
                         job_query="data", job_type="internship", language="en")
    
chdf1

15 new job postings retrieved from indeed. Stored in May-13-2021JS_test_results.xls.


,titles,companies,links,date_listed
0,Circular Economy Intern (100%)\nneu,Lafarge,https://ch.indeed.com/Stellen?q=data&jt=intern...,Heute
1,Middle Office Analyst - Internship\nneu,Total,https://ch.indeed.com/Stellen?q=data&jt=intern...,vor 1 Tag
2,CONTENT & COMMUNITY MANAGEMENT in the field of...,digitalswitzerland,https://ch.indeed.com/Stellen?q=data&jt=intern...,vor 1 Tag
3,R&D Internship - Product Development (Consumer...,Optotune,https://ch.indeed.com/Stellen?q=data&jt=intern...,vor 1 Tag
4,Data Intelligence & Automation Intern - Richem...,Richemont,https://ch.indeed.com/Stellen?q=data&jt=intern...,vor 2 Tagen
5,Intern- Battery technology\nneu,HILTI,https://ch.indeed.com/Stellen?q=data&jt=intern...,vor 2 Tagen
6,Student / Intern\nneu,Oracle,https://ch.indeed.com/Stellen?q=data&jt=intern...,vor 2 Tagen
7,Internship Risk Management\nneu,Credit Suisse Campus,https://ch.indeed.com/Stellen?q=data&jt=intern...,vor 2 Tagen
8,Systems Pharmacology Internship\nneu,LYO-X GmbH,https://ch.indeed.com/Stellen?q=data&jt=intern...,vor 2 Tagen
9,INTERN - ENVIRONMENT AFFAIRS [Temporary]\nneu,United Nations,https://ch.indeed.com/Stellen?q=data&jt=intern...,vor 2 Tagen


## download excel

In [13]:
from google.colab import files

if download_output:
    out_name = "search1.xlsx"
    chdf1.to_excel(out_name)
    files.download(out_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>